In [ ]:
# Import dependencies
import numpy as np
from citipy import citipy
from config import weather_api_key
import requests
import pandas as pd
import time

In [ ]:
# Generate 2,000 random lats/longs
lats = np.random.uniform(low=-90.000, high=90.000, size =2000)
longs = np.random.uniform(low=-90.000, high=90.000, size =2000)

# Store lat/long values in a list of tuples
lat_longs = zip(lats, longs)
coordinates = list(lat_longs)

In [ ]:
# Create a new list variable to store generated city values
cities = []

# Use citipy to identify the nearest cities
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    #Check to see if the city has already been found
    if city not in cities:
        cities.append(city)
        
# Print number of cities
print(f"The lat/long values generated {len(cities)} cities.")

In [ ]:
# Create a list that will store a dictionary of information for each city
city_data = []

# Store API base URL with API key to access weather data later
url = "http://api.openweathermap.org/data/2.5/weather?units=imperial&appid=" + weather_api_key

# Loop through each city
for i, city in enumerate(cities):
    
    # Create city url for API call
    # The city.replace() function replaces spaces in list of cities with pluses
    city_url = url + "&q=" + city.replace(" ","+")
    
    try:
        # Call the city data from the API
        print(f"Requesting data from {city}")
        city_weather = requests.get(city_url).json()
        time.sleep(1)
        
        # Parse out information for each city
        city_country = city_weather["sys"]["country"]
        city_lat = city_weather["coord"]["lat"]
        city_long = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_weather_description = city_weather["weather"][0]["description"]
        
        
        # Add city data to city dictionary
        city_data.append({"City": city.title(),
                         "Country": city_country,
                         "Lat": city_lat,
                         "Lng": city_long,
                         "Max Temp": city_max_temp,
                         "Humidity": city_humidity,
                         "Cloudiness": city_clouds,
                         "Wind Speed": city_wind,
                         "Current Description": city_weather_description})
    except:
        pass

In [ ]:
# Create a dataframe with the city data
city_data_df = pd.DataFrame(city_data)

# Store the dataframe in a csv file
output_data_file = "./WeatherPy_Database.csv"
city_data_df.to_csv(output_data_file, index_label="City_ID")